# Import Necessary Headers

In [10]:
import pandas as pd
import numpy as np
import shap
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load Updated CSV into Pandas Dataframe¶

In [2]:
# Hong et al. Dataset
df = pd.read_csv("hong_et_al/df_updt.csv")

In [3]:
# Drop unnecessary columns
df = df.drop(columns=df.filter(like='chrkidneydisease').columns)
df = df.drop(columns=df.filter(like='chrkidneydisease').columns)
df = df.drop(columns=df.filter(like='(pcp)screen').columns)
df = df.drop(columns=df.filter(regex='^cc_').columns)
df = df.dropna(subset=['egfr_CKD_EPI'])

# Impute median data for any remaining NaN values in numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Impute mode data for any remaining NaN values in categorical columns
categorical_cols = ['dep_name', 'gender', 'ethnicity', 'race', 'lang', 'religion',
                    'maritalstatus', 'employstatus', 'insurance_status', 'disposition', 'arrivalmode',
                    'arrivalmonth', 'arrivalday', 'arrivalhour_bin', 'previousdispo']

# One-hot encode categorical features
df = pd.get_dummies(df, columns=categorical_cols)

# Ensure no infinite values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(df.median(), inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212834 entries, 6 to 560485
Columns: 853 entries, esi to previousdispo_Transfer to Another Facility
dtypes: bool(100), float64(386), int64(367)
memory usage: 1.2 GB


# Define Train and Test Data Splits

In [5]:
# Exclude target variable from features frame
X = df.drop(columns=df.filter(like='egfr').columns)
y = df['egfr_CKD_EPI']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the numeric features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Ensure no NaN values in tensors
assert not X_train_tensor.isnan().any(), "NaN values found in X_train_tensor"
assert not y_train_tensor.isnan().any(), "NaN values found in y_train_tensor"

# Create DataLoader for batch processing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define Neural Network Architecture

In [6]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

input_dim = X_train.shape[1]
model = SimpleNN(input_dim)

# Training Loop

In [7]:
# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

Epoch 1/50, Loss: 378.8275669847651
Epoch 2/50, Loss: 164.56689022197529
Epoch 3/50, Loss: 125.94718625177256
Epoch 4/50, Loss: 110.03567397079267
Epoch 5/50, Loss: 93.53707371811721
Epoch 6/50, Loss: 87.41872566134263
Epoch 7/50, Loss: 83.33997437578536
Epoch 8/50, Loss: 80.28939761133492
Epoch 9/50, Loss: 75.43726235769243
Epoch 10/50, Loss: 74.76427598413512
Epoch 11/50, Loss: 70.76925529442708
Epoch 12/50, Loss: 68.64693345346204
Epoch 13/50, Loss: 68.10246276855469
Epoch 14/50, Loss: 66.63315330829356
Epoch 15/50, Loss: 64.60836514097011
Epoch 16/50, Loss: 63.311407531485976
Epoch 17/50, Loss: 60.82504834415589
Epoch 18/50, Loss: 59.66191792479138
Epoch 19/50, Loss: 57.62744734261279
Epoch 20/50, Loss: 56.718427256684876
Epoch 21/50, Loss: 55.088863367068505
Epoch 22/50, Loss: 53.147022518256875
Epoch 23/50, Loss: 51.89373515235531
Epoch 24/50, Loss: 50.232253775297366
Epoch 25/50, Loss: 48.588943937316685
Epoch 26/50, Loss: 47.05668720989378
Epoch 27/50, Loss: 45.817114049803266


# Evaluate Accuracy on Test Set

In [8]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader)}")

Test Loss: 14.890239196496683
